In [1]:
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time

In [2]:
db_connection = sqlite3.connect("DeepDao.db")
cursor = db_connection.cursor()

In [3]:
driver = webdriver.Firefox()
driver.get("https://deepdao.io/organizations")

def main_loop():
    try:
        for i in range(201, 203):
            if i >= 50:
                time.sleep(10)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(4)
            if i >=50 and i >=150:
                time.sleep(10)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(4)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(4)
                
            
            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, f'div.styles_tableRow__M0cR-:nth-child({i})'))
            ).click()
            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.styles_tabItem__2Wo4b:nth-child(2) > div:nth-child(1)"))
            ).click()

            # creates soup of finance page 
            soup = BeautifulSoup(driver.page_source, "html.parser")
            
            # inserts initial values into database
            dao_name = soup.find("h2", class_="styles_organizationName__14PZx").string
            treasury_aum = soup.find("h3", class_="styles_treasuryValue__3y4eO").string
            token_holders = soup.find("p", class_="styles_sum__1ReKM").string
            insert_vals = f"""
            INSERT INTO Governance (DAO, `Treasury AUM (usd)`, `Token Holders`) 
            VALUES (
                '{dao_name}',
                '{treasury_aum.replace('$', '')}',
                '{token_holders}'
            );
            """

            cursor.execute(insert_vals)
            
            
            #gets top 5 currencies held in DAO treasury
            get_currencies(soup, dao_name)
            
            #gets governance type description 
            get_governance_type(soup, dao_name)
            
            # Go to governance tab
            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.styles_tabItem__qqQD4:nth-child(2) > div:nth-child(1)"))
            ).click()
            # gets new soup for governance tab
            soup = BeautifulSoup(driver.page_source, "html.parser")
            
            # gets all proposal info
            get_governance(soup, dao_name)

            # commits all transactions to db
            db_connection.commit()
        
            # Go back to DAO listing
            time.sleep(30)
            driver.get("https://deepdao.io/organizations")
    except Exception as e:
        print(f'Failed on DAO: {dao_name}')
        print(f'Iteration: {i}')
        print(f'Reason: {e}')
        db_connection.close()


def get_currencies(soup, dao_name):
    currency_table = soup.find("div", class_="styles_rowsContainer__1NUzI")
    currency_info = currency_table.findChildren("div", class_="styles_tableRow__1H1CY")[0:5]
    final_data = []
    
    for coin in currency_info:
        data = coin.findChildren("span")
        final_data.append([i.string for i in data][1:])

    insert_dao = f"""
        INSERT INTO Treasury (DAO) 
        VALUES (
            '{dao_name}'
        );
        """
    cursor.execute(insert_dao)
    
    for i in range(0, len(final_data)):
        insert_vals = f"""
        UPDATE Treasury SET 
        `Top {i+1} Token (name)`= '{final_data[i][0]}',
        `Top {i+1} Token (usd)`= '{final_data[i][1].replace('$', '')}',
        `Top {i+1} Token (% of treasury)`= '{final_data[i][2].replace('%', '')}',
        `Top {i+1} Token (token balance)`= '{final_data[i][3]}'
        WHERE DAO='{dao_name}';
        """
        
        cursor.execute(insert_vals)
    db_connection.commit()
        
    

def get_governance(soup, dao_name):

    tables = soup.find_all("div", class_="styles_tableRow__37dw6")
   
    # activity table
    activity_data = tables[0].findChildren("div")
    activity_data_list = [i.string for i in activity_data]
    del activity_data_list[3]
    total_no_proposals = activity_data_list[0]
    proposal_creators = activity_data_list[1]
    success_rate = activity_data_list[2].replace('%', '')
    participation_rates = activity_data_list[3]

    # voting table
    voting_data = tables[1].findChildren("div")
    voting_data_list = [i.string for i in voting_data]
    avg_votes_per_proposal = voting_data_list[0]
    total_no_voters = voting_data_list[1]
    total_votes = voting_data_list[2]

    insert_vals = f"""
	UPDATE Governance SET
	`Total No. Proposals` = '{total_no_proposals}',
	`Proposal Creators` = '{proposal_creators}',
	`Success Rate (%)` = '{success_rate}',
	`Participation Rates (%)` = '{participation_rates}',
    `Total Votes` = '{total_votes}',
    `Total No. Voters` = '{total_no_voters}',
    `Avg voters per proposal` = '{avg_votes_per_proposal}'
	WHERE DAO='{dao_name}';
	"""
    cursor.execute(insert_vals)
    db_connection.commit()

def get_governance_type(soup, dao_name):
    governance_info = soup.find_all("div", class_="styles_description__8EqRx")
    governance_info_list = set([i.string for i in governance_info])

    off_chain = 0
    if 'Snapshot Governance' in governance_info_list:
        off_chain = 1
    
    for i in governance_info_list:
        if 'Governance' in i and 'Snapshot' not in i:
            on_chain = i
            break
        else:
            on_chain = 'Null'
    
    insert_vals = f"""
	UPDATE Governance SET
	`Off-Chain Governance (snapshot)` = '{off_chain}',
	`On-Chain Governance (type)` = '{on_chain}'
	WHERE DAO='{dao_name}';
	"""
    cursor.execute(insert_vals)
    db_connection.commit()
    

def get_top_voters(soup, dao_name):
    
    ...

main_loop()
db_connection.close()
driver.quit()


Failed on DAO: DAO Masters
Iteration: 202
Reason: Message: Failed to decode response from marionette

